In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from lightgbm import LGBMClassifier

In [ ]:
cl_train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Recognizance 2021/Round 1/client_train.csv",low_memory=False)
inv_train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Recognizance 2021/Round 1/train_invoice.csv",low_memory=False)
cl_test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Recognizance 2021/Round 1/client_test.csv",low_memory=False)
inv_test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Recognizance 2021/Round 1/test_invoice.csv",low_memory=False)

In [ ]:
def featengg(cl, inv):
    cl['client_catg'] = cl['client_catg'].astype('category')
    cl['disrict'] = cl['disrict'].astype('category')
    cl['region'] = cl['region'].astype('category')
    cl['region_group'] = cl['region'].apply(lambda x: 100 if x<100 else 300 if x>300 else 200)
    cl['creation_date'] = pd.to_datetime(cl['creation_date'])
    inv['counter_type'] = inv['counter_type'].map({"ELEC":1,"GAZ":0})
    inv['invoice_date'] = pd.to_datetime(inv['invoice_date'], dayfirst=True)
    inv['invoice_month'] = inv['invoice_date'].dt.month
    inv['invoice_year'] = inv['invoice_date'].dt.year
    inv['index_diff'] = inv['new_index'] - inv['old_index']
    return cl, inv

In [ ]:
cl_train, inv_train = featengg(cl_train, inv_train)
cl_test, inv_test = featengg(cl_test, inv_test)

In [ ]:
def agg_feature(invoice, client_df, agg_stat):
    full_df = pd.merge(client_df, invoice, on='client_id', how='left')
    return full_df

In [ ]:
agg_stat_columns = ['tarif_type','counter_number','counter_statue','counter_code','reading_remarque','consommation_level_1','consommation_level_2',
'consommation_level_3', 'consommation_level_4','old_index','new_index','months_number','counter_type', 'invoice_month','invoice_year','index_diff']

train = agg_feature(inv_train, cl_train, agg_stat_columns)
test = agg_feature(inv_test, cl_test, agg_stat_columns)

In [ ]:
train

In [ ]:
train.info()

In [ ]:
X=train.drop(['client_id', 'creation_date', 'target', 'invoice_date', 'counter_statue'], axis=1)
y=train['target']

In [ ]:
test

In [ ]:
x_test=test.drop(['client_id', 'creation_date', 'invoice_date', 'counter_statue'], axis=1)

In [ ]:
model = LGBMClassifier(random_state=42, n_estimators=800,num_leaves=450, max_depth=60,learning_rate=0.007)
model.fit(X,y)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.007, max_depth=60,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=800, n_jobs=-1, num_leaves=450, objective=None,
               random_state=42, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [ ]:
pred=model.predict(x_test)

,disrict,client_catg,region,region_group,tarif_type,counter_number,counter_code,reading_remarque,counter_coefficient,consommation_level_1,consommation_level_2,consommation_level_3,consommation_level_4,old_index,new_index,months_number,counter_type,invoice_month,invoice_year,index_diff,target
0,62,11,301,300,11,2165702163000,207,9,1,607,0,0,0,211,818,4,1,3,2018,607,0.0
1,62,11,301,300,11,2165702163000,207,9,1,2,0,0,0,209,211,4,1,11,2017,2,0.0
2,62,11,301,300,11,2165702163000,207,9,1,689,0,0,0,1706,2395,4,1,3,2019,689,0.0
3,62,11,301,300,11,2165702163000,207,9,1,212,0,0,0,818,1030,4,1,10,2018,212,0.0
4,62,11,301,300,11,2165702163000,207,9,1,676,0,0,0,1030,1706,4,1,11,2018,676,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1197049,60,11,101,200,10,1253571,202,9,1,400,135,0,0,3197,3732,8,1,8,2005,535,0.0
1197050,60,11,101,200,10,1253571,202,6,1,200,6,0,0,3732,3938,4,1,12,2005,206,0.0
1197051,60,11,101,200,11,560948,203,6,1,259,0,0,0,13884,14143,4,1,9,1996,259,0.0
1197052,60,11,101,200,11,560948,203,6,1,603,0,0,0,13281,13884,4,1,5,1996,603,0.0


In [ ]:
test['target']=pred
test

,disrict,client_id,client_catg,region,creation_date,region_group,invoice_date,tarif_type,counter_number,counter_statue,counter_code,reading_remarque,counter_coefficient,consommation_level_1,consommation_level_2,consommation_level_3,consommation_level_4,old_index,new_index,months_number,counter_type,invoice_month,invoice_year,index_diff,target
0,62,train_Client_675,11,301,2017-04-22,300,2018-03-14,11,2165702163000,0,207,9,1,607,0,0,0,211,818,4,1,3,2018,607,0.0
1,62,train_Client_675,11,301,2017-04-22,300,2017-11-14,11,2165702163000,0,207,9,1,2,0,0,0,209,211,4,1,11,2017,2,0.0
2,62,train_Client_675,11,301,2017-04-22,300,2019-03-14,11,2165702163000,0,207,9,1,689,0,0,0,1706,2395,4,1,3,2019,689,0.0
3,62,train_Client_675,11,301,2017-04-22,300,2018-10-07,11,2165702163000,0,207,9,1,212,0,0,0,818,1030,4,1,10,2018,212,0.0
4,62,train_Client_675,11,301,2017-04-22,300,2018-11-15,11,2165702163000,0,207,9,1,676,0,0,0,1030,1706,4,1,11,2018,676,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1197049,60,train_Client_99998,11,101,1993-12-22,200,2005-08-19,10,1253571,0,202,9,1,400,135,0,0,3197,3732,8,1,8,2005,535,0.0
1197050,60,train_Client_99998,11,101,1993-12-22,200,2005-12-19,10,1253571,0,202,6,1,200,6,0,0,3732,3938,4,1,12,2005,206,0.0
1197051,60,train_Client_99999,11,101,1986-02-18,200,1996-09-25,11,560948,0,203,6,1,259,0,0,0,13884,14143,4,1,9,1996,259,0.0
1197052,60,train_Client_99999,11,101,1986-02-18,200,1996-05-28,11,560948,0,203,6,1,603,0,0,0,13281,13884,4,1,5,1996,603,0.0


In [ ]:
tst =test.groupby(['client_id'],as_index=False).mean()
tst

,client_id,region_group,tarif_type,counter_number,counter_statue,counter_code,reading_remarque,counter_coefficient,consommation_level_1,consommation_level_2,consommation_level_3,consommation_level_4,old_index,new_index,months_number,counter_type,invoice_month,invoice_year,index_diff,target
0,train_Client_675,300.0,11.0,2.165702e+12,0.0,207.0,9.0,1.0,607.0,0.0,0.0,0.0,818.0,1030.0,4.0,1.0,7.0,2018.0,607.0,0.0
1,train_Client_6750,300.0,11.0,1.013270e+05,0.0,203.0,6.0,1.0,202.0,0.0,0.0,0.0,1034.0,1275.0,4.0,1.0,4.0,2015.0,202.0,0.0
2,train_Client_67500,300.0,25.0,1.058270e+05,0.0,103.5,6.0,1.0,200.0,0.0,0.0,0.0,5059.5,5188.0,4.0,0.5,8.0,2011.0,298.0,0.0
3,train_Client_67501,200.0,11.0,1.292196e+06,0.0,203.0,8.0,1.0,605.0,0.0,0.0,0.0,2847.0,2978.0,4.0,1.0,5.0,2012.0,605.0,0.0
4,train_Client_67502,300.0,11.0,1.141938e+06,0.0,203.0,9.0,1.0,820.0,0.0,0.0,0.0,34115.5,34941.5,4.0,1.0,4.0,2013.5,1118.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36105,train_Client_99995,300.0,11.0,8.972860e+05,0.0,207.0,6.0,1.0,0.0,0.0,0.0,0.0,14.0,14.0,4.0,1.0,6.0,2012.0,0.0,0.0
36106,train_Client_99996,300.0,40.0,3.987830e+05,0.0,5.0,9.0,1.0,121.0,0.0,0.0,0.0,556.0,667.0,4.0,0.0,6.0,2016.0,121.0,0.0
36107,train_Client_99997,300.0,11.0,2.621950e+05,0.0,207.0,9.0,1.0,332.0,0.0,0.0,0.0,1420.5,1749.0,4.0,1.0,7.0,2014.5,332.0,0.0
36108,train_Client_99998,200.0,10.0,1.253571e+06,0.0,202.0,7.5,1.0,300.0,70.5,0.0,0.0,3464.5,3835.0,6.0,1.0,10.0,2005.0,370.5,0.0
